In [1]:
# Initializing imports.
from pathlib import Path
import os
import zipfile
import requests,io
import pandas as pd
import datetime as dt

In [2]:
# 
Divvy_Trips_2019_Q1_link = 'https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q1.zip'
Divvy_Trips_2019_Q2_link = 'https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q2.zip'
Divvy_Trips_2019_Q3_link = 'https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q3.zip'
Divvy_Trips_2019_Q4_link = 'https://divvy-tripdata.s3.amazonaws.com/Divvy_Trips_2019_Q4.zip'

In [3]:
# 
getfolder1 = requests.get(Divvy_Trips_2019_Q1_link)
getfolder2 = requests.get(Divvy_Trips_2019_Q2_link)
getfolder3 = requests.get(Divvy_Trips_2019_Q3_link)
getfolder4 = requests.get(Divvy_Trips_2019_Q4_link)

In [4]:
#
unzipfolder1 = zipfile.ZipFile(io.BytesIO(getfolder1.content))
unzipfolder2 = zipfile.ZipFile(io.BytesIO(getfolder2.content))
unzipfolder3 = zipfile.ZipFile(io.BytesIO(getfolder3.content))
unzipfolder4 = zipfile.ZipFile(io.BytesIO(getfolder4.content))

In [5]:
#
unzipfolder1.extractall('./Q1/Divvy_Trips_2019_Q1')
unzipfolder2.extractall('./Q2/Divvy_Trips_2019_Q2')
unzipfolder3.extractall('./Q3/Divvy_Trips_2019_Q3')
unzipfolder4.extractall('./Q4/Divvy_Trips_2019_Q4')

In [6]:
# 
os.rename("./Q1/Divvy_Trips_2019_Q1/Divvy_Trips_2019_Q1","./Q1/Divvy_Trips_2019_Q1/Divvy_Trips_2019_Q1.csv")
os.rename("./Q2/Divvy_Trips_2019_Q2/Divvy_Trips_2019_Q2","./Q2/Divvy_Trips_2019_Q2/Divvy_Trips_2019_Q2.csv")

In [7]:
# 
Q1_df = pd.read_csv('./Q1/Divvy_Trips_2019_Q1/Divvy_Trips_2019_Q1.csv')
Q2_df = pd.read_csv('./Q2/Divvy_Trips_2019_Q2/Divvy_Trips_2019_Q2.csv')
Q3_df = pd.read_csv('./Q3/Divvy_Trips_2019_Q3/Divvy_Trips_2019_Q3.csv')
Q4_df = pd.read_csv('./Q4/Divvy_Trips_2019_Q4/Divvy_Trips_2019_Q4.csv')

In [8]:
# 
Q2_df = Q2_df.rename(
    columns={'01 - Rental Details Rental ID' : 'trip_id'
             , '01 - Rental Details Local Start Time' : 'start_time'
             ,'01 - Rental Details Local End Time' : 'end_time'
             ,'01 - Rental Details Bike ID' : 'bikeid'
             ,'01 - Rental Details Duration In Seconds Uncapped' : 'tripduration'
             ,'03 - Rental Start Station ID' : 'from_station_id'
             ,'03 - Rental Start Station Name' : 'from_station_name'
             ,'02 - Rental End Station ID' : 'to_station_id'
             ,'02 - Rental End Station Name' : 'to_station_name'
             ,'User Type' : 'usertype'
             ,'Member Gender' : 'gender'
             ,'05 - Member Details Member Birthday Year' : 'birthyear'}
)

In [9]:
# 
joined_Divvy_data = Q1_df.append([Q2_df,Q3_df,Q4_df], ignore_index=True, sort=False)

In [10]:
# 
joined_Divvy_data.drop(columns=[
    'trip_id'
    , 'end_time', 'bikeid'
    , 'from_station_id'
    , 'from_station_name'
    , 'to_station_id'
    ,'to_station_name'], inplace=True)

In [11]:
# 
joined_Divvy_data_split = joined_Divvy_data['start_time'].str.split(" ", n = 1, expand = True)

In [12]:
# 
joined_Divvy_data_split = joined_Divvy_data_split.rename(columns={0:'Date'})

In [13]:
# 
joined_Divvy_data_split.drop(columns=[1], inplace=True)

In [14]:
# 
joined_Divvy_data_split['Date']=pd.to_datetime(joined_Divvy_data_split.Date)

In [15]:
# 
joined_Divvy_data_split['Date']=joined_Divvy_data_split['Date'].dt.strftime('%m/%d/%Y')

In [16]:
# 
joined_Divvy_data_all = pd.concat([joined_Divvy_data, joined_Divvy_data_split], axis="columns", join="inner")

In [17]:
# 
joined_Divvy_data_all.drop(columns=['start_time'], inplace=True)

In [18]:
# 
joined_Divvy_data_all = joined_Divvy_data_all.rename(columns={'usertype':'User_Type', 'tripduration':'Trip_Duration_min', 'gender':'Gender', 'birthyear':'Birth_Year'})

In [19]:
# Cleansing nulls from Joined Dataframe by replacing NAN values with empty strings.
Full_Divvy_data = joined_Divvy_data_all.fillna("")

In [20]:
# 
Full_Divvy_data = Full_Divvy_data[['Date', 'User_Type', 'Trip_Duration_min', 'Gender', 'Birth_Year']]
Full_Divvy_data.head()

,Date,User_Type,Trip_Duration_min,Gender,Birth_Year
0,01/01/2019,Subscriber,390.0,Male,1989.0
1,01/01/2019,Subscriber,441.0,Female,1990.0
2,01/01/2019,Subscriber,829.0,Female,1994.0
3,01/01/2019,Subscriber,"1,783.0",Male,1993.0
4,01/01/2019,Subscriber,364.0,Male,1994.0


In [21]:
# 
Total_2019_Users = joined_Divvy_data_all.groupby('Date').User_Type.count().to_frame().rename(columns={'User_Type':'Total_2019_Users'}).reset_index()
Total_2019_Users.head()

,Date,Total_2019_Users
0,01/01/2019,1312
1,01/02/2019,4170
2,01/03/2019,5618
3,01/04/2019,6793
4,01/05/2019,6179


In [22]:
# 
Weeks_2019 = pd.date_range('2018-12-31', '2019-12-30', freq='D').to_series()

In [23]:
# 
Weeks_2019 = Weeks_2019.dt.isocalendar().week

In [24]:
# 
Weeks_2019_df = Weeks_2019.to_frame().reset_index().drop(columns=['index']).rename(columns={'week':'Week'})

In [25]:
# 
Weeks_2019_df.iloc[[364],['Week']]=52

In [26]:
# 
Weeks_2019_df.loc[len(Weeks_2019_df.index)]=52

In [27]:
# 
Weeks_2019_df = Weeks_2019_df.drop(labels=0, axis=0)

In [28]:
# 
Weeks_2019_df = Weeks_2019_df.reset_index().drop(columns=['index'])

In [29]:
# 
Weekly_df = pd.concat([Weeks_2019_df, Total_2019_Users], axis="columns", join="inner")
Weekly_df

,Week,Date,Total_2019_Users
0,1,01/01/2019,1312
1,1,01/02/2019,4170
2,1,01/03/2019,5618
3,1,01/04/2019,6793
4,1,01/05/2019,6179
...,...,...,...
360,52,12/27/2019,3981
361,52,12/28/2019,2768
362,52,12/29/2019,3902
363,52,12/30/2019,3308


In [30]:
# 
Weekly_Divvy_Users_df = Weekly_df.groupby('Week').Total_2019_Users.sum().reset_index()

In [31]:
# 
Weekly_Divvy_Users_df

,Week,Total_2019_Users
0,1,27749
1,2,30930
2,3,28005
3,4,13994
4,5,8871
5,6,26529
6,7,22820
7,8,23143
8,9,26498
9,10,24766


In [54]:
# 
Weekly_Divvy_Users_df.to_csv(r'../Weekly_Divvy_Users.csv', index=False)

In [33]:
# 
Divvy_data = joined_Divvy_data_all
Divvy_data.dropna(inplace=True)

In [34]:
# 
Divvy_data['Birth_Year'] = Divvy_data['Birth_Year'].astype(int)

In [35]:
# 
Divvy_data['Trip_Duration_min']=Divvy_data['Trip_Duration_min'].str.replace('.,','')

<ipython-input-35-7240950e9bbd>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  Divvy_data['Trip_Duration_min']=Divvy_data['Trip_Duration_min'].str.replace('.,','')


In [36]:
# 
Divvy_data['Trip_Duration_min'] = Divvy_data['Trip_Duration_min'].astype(float).astype(int)

In [37]:
# 
Divvy_data['Trip_Duration_min'] = Divvy_data['Trip_Duration_min']/60

In [38]:
# 
Divvy_data['Trip_Duration_min'] = Divvy_data['Trip_Duration_min'].round(1).astype(float)

In [39]:
Divvy_data['Date'] = pd.to_datetime(Divvy_data.Date)

In [40]:
# 
Divvy_data = Divvy_data[['Date', 'User_Type', 'Trip_Duration_min', 'Gender', 'Birth_Year']]
Divvy_data.head()

,Date,User_Type,Trip_Duration_min,Gender,Birth_Year
0,2019-01-01,Subscriber,6.5,Male,1989
1,2019-01-01,Subscriber,7.4,Female,1990
2,2019-01-01,Subscriber,13.8,Female,1994
3,2019-01-01,Subscriber,13.0,Male,1993
4,2019-01-01,Subscriber,6.1,Male,1994


In [41]:
# 
Divvy_data

,Date,User_Type,Trip_Duration_min,Gender,Birth_Year
0,2019-01-01,Subscriber,6.5,Male,1989
1,2019-01-01,Subscriber,7.4,Female,1990
2,2019-01-01,Subscriber,13.8,Female,1994
3,2019-01-01,Subscriber,13.0,Male,1993
4,2019-01-01,Subscriber,6.1,Male,1994
...,...,...,...,...,...
3817999,2019-12-31,Subscriber,2.9,Male,1981
3818000,2019-12-31,Subscriber,8.9,Male,1992
3818001,2019-12-31,Subscriber,8.7,Male,1967
3818002,2019-12-31,Subscriber,8.6,Female,1970


In [42]:
# 
Divvy_data.dtypes

Date                 datetime64[ns]
User_Type                    object
Trip_Duration_min           float64
Gender                       object
Birth_Year                    int32
dtype: object

In [43]:
# 


In [44]:
# 


In [45]:
# 


In [46]:
#


In [47]:
#


In [48]:
# 


In [49]:
# 


In [50]:
# 


In [51]:
# 


In [52]:
# 


In [53]:
# 
